In [1]:
from datasets import load_from_disk
dataset = load_from_disk('../eval_cas')
dev = dataset['dev']
test = dataset['test']

In [2]:
from transformers import pipeline

translator = pipeline("translation", model="finetune_mt_model/checkpoint-163000")

/home/hawau.toyin/.conda/envs/speech/lib/python3.10/site-packages/transformers/pipelines/__init__.py:958: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [3]:
test

Dataset({
    features: ['id', 'asr', 'target'],
    num_rows: 14759
})

In [3]:
dev

Dataset({
    features: ['id', 'asr', 'target'],
    num_rows: 14759
})

In [4]:
def predict(batch):
    pred = translator(batch['asr'])
    batch['mt'] = pred[0]['translation_text']
    return batch

In [5]:
test_result = test.map(predict)

Loading cached processed dataset at /home/hawau.toyin/Documents/eval_cas/test/cache-0af5319a1f86720c.arrow


In [10]:
dev_result = dev.map(predict)

Loading cached processed dataset at /home/hawau.toyin/Documents/eval_cas/dev/cache-5cd7b980fcc03e6c.arrow


In [11]:
dev_result[2]

{'id': 'common_voice_fr_17300275',
 'asr': "translate French to English: je retire l'amendement numéro cinq cent cinquante-cinq",
 'target': 'i withdraw amendment number one hundred and fifty five',
 'mt': 'i withdraw the amendment number five hundred fifty five'}

In [6]:
test_result[2]

{'id': 'common_voice_fr_17299401',
 'asr': 'translate French to English: pas si nombreuses que ça',
 'target': 'not that many',
 'mt': 'not so many'}

In [7]:
from evaluate import load

In [8]:
wer = load("wer")

In [9]:
print(wer.compute(predictions=test_result["mt"], references=test_result["target"]))

0.8650950495467301


In [11]:
print(wer.compute(predictions=dev_result["mt"], references=dev_result["target"]))

0.9507153070682829
